In [1]:
import whisperx
from pyannote.audio import Pipeline
from pydub import AudioSegment  
import os
import pandas as pd

from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv('hf_token')



def transcribe_with_diarization(audio_file_path,OutputFile, model_type="large-v2", device="cpu", hf_token=hf_token):
    # Step 1: Load the WhisperX model with float32 compute type
    model = whisperx.load_model(model_type, device, compute_type="float32")

    # Step 2: Load the audio file
    audio = whisperx.load_audio(audio_file_path)

    # Step 3: Transcribe audio
    result = model.transcribe(audio)

    # Step 4: Load an alignment model for diarization
    align_model, align_metadata = whisperx.load_align_model(language_code=result["language"], device=device)

    # Step 5: Align Whisper output
    result = whisperx.align(result["segments"], align_model, align_metadata, audio, device, return_char_alignments=False)

    # Step 6: Perform speaker diarization
    diarize_model = whisperx.DiarizationPipeline(use_auth_token=hf_token, device=device)
    diarize_segments = diarize_model(audio)

    # Step 7: Assign speaker IDs to word-level segments
    result = whisperx.assign_word_speakers(diarize_segments, result)

    transcription = result["segments"]


    return transcription

C:\Users\hamza\AppData\Local\Programs\Python\Python310\lib\site-packages\pyannote\audio\core\io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
C:\Users\hamza\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_audiomentations\utils\io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
torchvision is not available - cannot save figures
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [ ]:
transcription =  transcribe_with_diarization("Audio.wav","fle")

print(transcription)


tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]